# Create and test metadata_handler

In [1]:
import src.metadata_handler as mdh

%autoreload 2

In [2]:
from iptcinfo import IPTCInfo
from iptcinfo import IPTCData
import json

from pymongo import MongoClient

from iptcinfo import c_datasets, c_datasets_r

import os
import pandas as pd
import numpy as np

import boto3

In [3]:
a_photo = 'first_sample/EK000025-2.JPG'
info = IPTCInfo(a_photo)
data_dict = dict(info.data)
info_field_nums = set(data_dict.keys())
test_set = set(['a', 'b', 'c'])

('WARNING: problems with charset recognition', "'\\x1b'")


In [4]:
# rework get_fields to 
def get_fields(directory):
    ## strip '/' only if it is at the end
    #  otherwise it may mess with nested dirs
    #  (e.g. sample/photo/set)
    # directory = directory.strip('/')
    field_set = set()
    for p, dirs, files in os.walk(directory):
        for ff in files:
            if ff[-4:] == '.JPG':
                info = IPTCInfo(directory+'/'+ ff)
                data_dict = dict(info.data)
                fields = set(data_dict.keys())
                field_set = field_set.union(fields)                
    return field_set


In [5]:
fields = set([])
fields = get_fields('first_sample/')
fields

('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: pro

{5, 20, 25, 55, 60, 80, 92, 116, 118, 120}

# SQL direct from photo?

Signature: info.exportSQL(tablename, mappings, extra)
Docstring:
statement = info.exportSQL('mytable', mappings, extra-data)

mappings = {
    'IPTC dataset name here': 'your table column name here',
    'caption/abstract': 'caption',
    'city': 'city',
    'province/state': 'state} # etc etc etc.

Returns a SQL statement to insert into your given table name a set
of values from the image. Caller passes in a reference to a hash
which maps IPTC dataset names into column names for the database
table. Optionally pass in a ref to a hash of extra data which will
also be included in the insert statement. Keys in that hash must
be valid column names.

### Create SQL table

```
<table name>
    |
    |- <col name = field name>
    |- ...
    |- ...
    |
    |- <some way to reference/retrieve photo
```

### Use IPTCInfo to import metadata

```python
for photo in photos:
    info = IPTCInfo(photo)
    info.exportSQL('<table name>', map{})
```    

# JSON from metadata

In [6]:
def set_to_dict(field_set):
    field_dict = {}
    for field in field_set:
        field_dict[field] = c_datasets[field]
    return field_dict

In [21]:
field_dict = set_to_dict(fields)
field_dict

{5: 'object name',
 20: 'supplemental category',
 25: 'keywords',
 55: 'date created',
 60: 'time created',
 80: 'by-line',
 92: 'sub-location',
 116: 'copyright notice',
 118: 'contact',
 120: 'caption/abstract'}

In [22]:


def Mdata_dictionary(photo):
    ''' get a photos metadata as dict '''

    # return relevant data for a photo
    dd = {}
    info = IPTCInfo(photo)
    data_dict = dict(info.data)         
    for field in field_dict.keys():
        if data_dict.has_key(field):
            # print field_dict[field],' : ', data_dict[field]
            dd[field_dict[field]] = data_dict[field]
        else:
            print field
    return dd

def Mdata_to_json(photo, outfile):
    with open(outfile, 'a') as outf:
        json.dump(Mdata_dictionary(photo), outf, indent = 4)


In [268]:
# Mdata_to_json('first_sample/EK000004.JPG', 'test.json')

In [269]:
# field_dict

# Data Base!

In [273]:
db = pd.read_json('data/test_DB.json')

In [275]:
db.head().T

,0,1,2,3,4
by-line,Cathy Clark,Picasa,Cathy Clark,Picasa,Cathy Clark
caption/abstract,M2E32L107-108R390B311,NaN,1,NaN,1
contact,[],[],[],[],[]
copyright notice,Conservation Northwest,NaN,Conservation Northwest,NaN,Conservation Northwest
date created,2.01505e+07,NaN,2.01506e+07,NaN,2.01201e+07
file_path,first_sample/EK000004-2.JPG,first_sample/EK000004.JPG,first_sample/EK000010-2.JPG,first_sample/EK000010.JPG,first_sample/EK000011-2.JPG
keywords,[Camera Check],[camera check],[hoary marmot],[marmot],[hoary marmot]
object name,wolverine,NaN,wolverine,NaN,wolverine
sub-location,Blackjack Ridge-2015-1,NaN,Blackjack Ridge-2015-1,NaN,Blackjack Ridge-2015-1
supplemental category,[],[],[],[],[]


# MongoDB conversion:

```python
from pymongo import MongoClient
# connect to MongoDB client
client = MongoClient()
# connect to MongoDB database
db = prod_client['random_database']
# connect to MongoDB collections
collection = db['random_collection']
# insert document into collection
collection.insert_one({'random data': data_variable})
```

In [9]:
mc = MongoClient()
mongo_db = mc['cnw']
collection = mongo_db['mongo_test']

In [10]:
photo_dict = Mdata_dictionary(a_photo)

('WARNING: problems with charset recognition', "'\\x1b'")
999
{'date created': '20150610', 'copyright notice': 'Conservation Northwest', 'sub-location': 'Blackjack Ridge-2015-1', 'by-line': 'Cathy Clark', 'supplemental category': [], 'contact': [], 'time created': '070424', 'keywords': ['mule deer'], 'object name': 'wolverine', 'caption/abstract': '1'}


In [12]:
collection.insert_one(photo_dict)

In [18]:
def create_mongoDB(directory, database_name = 'cnw', col_name = 'metadata'):
    # For final version change default database_name to 'Wildlife_Identification'
    mc = MongoClient()
    mongo_db = mc[database_name]
    collection = mongo_db[col_name]
    for p, dirs, files in os.walk(directory):
        for ff in files:
            if ff[-4:] == '.JPG':
                # metadata = build_dictionary(ff)
                metadata = Mdata_dictionary(p+'/'+ff)

In [19]:
!ls

1_metadata_handler.ipynb create_EC2.ipynb         sample.json
3_AWS.ipynb              data                     scratchwork
ProjectNotes.md          first_sample             src
README.md                raw_DB.json              temp_photo.jpg


In [23]:
mongo_metadata = create_mongoDB('first_sample')

('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
5
60
116
55
120
92
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
5
60
116
55
120
92
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
5
60
116
55
120
92
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
5
60
116
55
120
92
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
5
60
116
55
120
92
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
5
60
116
55
120
92
('WARNING: problems with charset recognition', "'\\x1b'")
('WARNING: problems with charset recognition', "'\\x1b'")
5
60
116
55
120
92
('WARNING: problems with charset recognition', "'\\x1b'